In [11]:
import os
import shutil
import random
import math

# 1. 기본 경로 설정
# Windows 경로 사용 시, 역슬래시(\) 문제를 피하기 위해 문자열 앞에 r을 붙입니다.
base_path = r"C:\test"

print(f"기준 폴더: {base_path}")
print("데이터셋 구성을 시작합니다...")

# 2. 'Computer Vision Lab' 안의 모든 항목 목록을 가져옵니다.
all_items = os.listdir(base_path)
food_folders = [item for item in all_items if os.path.isdir(os.path.join(base_path, item))]
food_folders

output_dirs = {'train', 'valid', 'test'}
food_folders = [folder for folder in food_folders if folder not in output_dirs]

food_folders

기준 폴더: C:\test
데이터셋 구성을 시작합니다...


['chicken', 'pizza', 'ramen']

In [17]:
import os
import shutil
import random
import math

# 1. 기본 경로 설정
# Windows 경로 사용 시, 역슬래시(\) 문제를 피하기 위해 문자열 앞에 r을 붙입니다.
base_path = r"C:\test"

print(f"기준 폴더: {base_path}")
print("데이터셋 구성을 시작합니다...")

try:
    # 2. 'Computer Vision Lab' 안의 모든 항목 목록을 가져옵니다.
    all_items = os.listdir(base_path)

    # 3. 폴더만 필터링합니다. (zip, DS_STORE 같은 파일 제외)
    food_folders = [item for item in all_items if os.path.isdir(os.path.join(base_path, item))]
    
    # 4. 스크립트가 생성할 train, valid, test 폴더는 처리 대상에서 제외합니다.
    #    (스크립트를 여러 번 실행해도 안전하도록)
    output_dirs = {'train', 'valid', 'test'}
    food_folders = [folder for folder in food_folders if folder not in output_dirs]

    print(f"\n처리 대상 음식 폴더: {food_folders}\n")

    # 5. 각 음식 폴더를 순회하며 작업 수행
    for food_name in food_folders:
        print(f"--- '{food_name}' 폴더 처리 중 ---")
        
        source_dir = os.path.join(base_path, food_name, 'png')

        # 'png' 폴더가 있는지 확인
        if not os.path.isdir(source_dir):
            print(f"경고: '{source_dir}' 경로에 png 폴더가 없습니다. 건너뜁니다.")
            continue

        # 'png' 폴더 안의 모든 파일 목록을 가져옴
        images = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
        
        if not images:
            print(f"경고: '{source_dir}' 폴더에 이미지가 없습니다. 건너뜁니다.")
            continue
            
        # 6. 이미지 목록을 무작위로 섞음
        random.shuffle(images)
        print("==========================",images)
        
        # 7. 7:1.5:1.5 비율로 분할 지점 계산
        total_count = len(images)
        train_count = math.floor(total_count * 0.7)
        valid_count = math.floor(total_count * 0.15)
        # test_count는 나머지 전체로 할당하여 소수점 오차로 인한 파일 누락 방지

        train_files = images[:train_count]
        valid_files = images[train_count : train_count + valid_count]
        test_files = images[train_count + valid_count :]
        
        print(f"총 {total_count}개 이미지 -> Train: {len(train_files)}, Valid: {len(valid_files)}, Test: {len(test_files)}")

        # 8. 파일을 이동시킬 목적지 폴더 경로 설정 및 생성
        dest_train_dir = os.path.join(base_path, 'train', food_name)
        dest_valid_dir = os.path.join(base_path, 'valid', food_name)
        dest_test_dir = os.path.join(base_path, 'test', food_name)
        
        os.makedirs(dest_train_dir, exist_ok=True)
        os.makedirs(dest_valid_dir, exist_ok=True)
        os.makedirs(dest_test_dir, exist_ok=True)

        # 9. 파일 이동 함수
        def move_files(files, destination_dir):
            for f in files:
                source_file = os.path.join(source_dir, f)
                dest_file = os.path.join(destination_dir, f)
                shutil.copy2(source_file, dest_file)

        # 파일 이동 실행
        move_files(train_files, dest_train_dir)
        move_files(valid_files, dest_valid_dir)
        move_files(test_files, dest_test_dir)
        
        print(f"'{food_name}' 폴더의 이미지 이동 완료.\n")

    print("모든 작업이 성공적으로 완료되었습니다!")

except FileNotFoundError:
    print(f"오류: 기본 경로 '{base_path}'를 찾을 수 없습니다. 경로를 확인해주세요.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

기준 폴더: C:\test
데이터셋 구성을 시작합니다...

처리 대상 음식 폴더: ['chicken', 'pizza', 'ramen']

--- 'chicken' 폴더 처리 중 ---
========================== ['chicken002.png', 'chicken001.png']
총 2개 이미지 -> Train: 1, Valid: 0, Test: 1
'chicken' 폴더의 이미지 이동 완료.

--- 'pizza' 폴더 처리 중 ---
========================== ['pizza002.png', 'pizza001.png']
총 2개 이미지 -> Train: 1, Valid: 0, Test: 1
'pizza' 폴더의 이미지 이동 완료.

--- 'ramen' 폴더 처리 중 ---
========================== ['ramen001.png', 'ramen002.png']
총 2개 이미지 -> Train: 1, Valid: 0, Test: 1
'ramen' 폴더의 이미지 이동 완료.

모든 작업이 성공적으로 완료되었습니다!


In [4]:
import os
import pandas as pd
import glob
from sklearn.model_selection import train_test_split

# 1. 모든 클래스 폴더가 있는 단일 원본 경로만 지정합니다.
main_data_dir = r'C:\test'

# 2. 전체 파일 경로와 레이블을 DataFrame으로 만듭니다.
print("Creating file list...")
all_filepaths = glob.glob(os.path.join(main_data_dir, '*', 'png', '*.png'))
all_labels = [os.path.basename(os.path.dirname(os.path.dirname(p))) for p in all_filepaths]

df = pd.DataFrame({'filepath': all_filepaths, 'label': all_labels})
print(f"Total images found: {len(df)}")

# 3. train_test_split을 사용해 데이터를 70:15:15 비율로 분할합니다.
# 먼저 train(70%)과 temp(30%)로 나눕니다.
train_df, temp_df = train_test_split(
    df,
    train_size=0.7,
    shuffle=True,
    stratify=df['label'],
    random_state=42
)

# temp(30%)를 valid(15%)와 test(15%)로 나눕니다. (30%의 50%는 15%)
valid_df, test_df = train_test_split(
    temp_df,
    train_size=0.5,
    shuffle=True,
    stratify=temp_df['label'],
    random_state=42
)

Creating file list...
Total images found: 24


In [5]:
valid_df

,filepath,label
14,C:\test\pizza\png\pizza002 - 복사본.png,pizza
23,C:\test\ramen\png\ramen002.png,ramen
18,C:\test\ramen\png\ramen001 - 복사본.png,ramen
6,C:\test\chicken\png\chicken002 - 복사본.png,chicken


In [6]:
df

,filepath,label
0,C:\test\chicken\png\chicken001 - 복사본 (2).png,chicken
1,C:\test\chicken\png\chicken001 - 복사본 - 복사본.png,chicken
2,C:\test\chicken\png\chicken001 - 복사본.png,chicken
3,C:\test\chicken\png\chicken001.png,chicken
4,C:\test\chicken\png\chicken002 - 복사본 (2).png,chicken
5,C:\test\chicken\png\chicken002 - 복사본 - 복사본.png,chicken
6,C:\test\chicken\png\chicken002 - 복사본.png,chicken
7,C:\test\chicken\png\chicken002.png,chicken
8,C:\test\pizza\png\pizza001 - 복사본 (2).png,pizza
9,C:\test\pizza\png\pizza001 - 복사본 - 복사본.png,pizza


In [7]:
len(train_df)

16

In [ ]:
햐